<h1 align="center"><font color='lavander'>DIGIT RECOGNISER</font></h1>

<h2 align="left"><font color='gray'>Import necessary libraries 📚</font></h2>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

<h2 align="left"><font color='gray'>Import the  dataset  💾
 </font></h2>

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


<h2 align="left"><font color='gray'>EDA - Exploratary Data Analysis  ⚒️
</font></h2>

In [3]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [4]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train.shape

(42000, 785)

In [6]:
train_x = train.drop(['label'],axis = 1).astype('int32')
train_y = train['label'].astype('float32')
test_x = test.astype('float32')

In [7]:
train_x = train_x.values.reshape(-1,28,28,1)
train_x = train_x/255.0
test_x = test_x.values.reshape(-1,28,28,1)
test_x = test_x/255.0

In [8]:
train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

In [9]:
train_y

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

<h2 align="left"><font color='gray'>Model 
 </font></h2>

In [10]:
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='Same'),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='Same'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
    
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='Same'),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='Same'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                    

In [11]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy',metrics=['accuracy'])
model.fit(train_x,train_y,batch_size=50,epochs=10)

Epoch 1/10
840/840 [==============================] - 57s 66ms/step - loss: 0.1950 - accuracy: 0.9377
Epoch 2/10
840/840 [==============================] - 54s 64ms/step - loss: 0.0573 - accuracy: 0.9829
Epoch 3/10
840/840 [==============================] - 54s 65ms/step - loss: 0.0435 - accuracy: 0.9867
Epoch 4/10
840/840 [==============================] - 54s 65ms/step - loss: 0.0352 - accuracy: 0.9891
Epoch 5/10
840/840 [==============================] - 55s 65ms/step - loss: 0.0310 - accuracy: 0.9905
Epoch 6/10
840/840 [==============================] - 55s 66ms/step - loss: 0.0258 - accuracy: 0.9923
Epoch 7/10
840/840 [==============================] - 54s 64ms/step - loss: 0.0239 - accuracy: 0.9923
Epoch 8/10
840/840 [==============================] - 55s 65ms/step - loss: 0.0210 - accuracy: 0.9936
Epoch 9/10
840/840 [==============================] - 54s 65ms/step - loss: 0.0189 - accuracy: 0.9944
Epoch 10/10
840/840 [==============================] - 55s 65ms/step - loss: 0.015

<h2 align="left"><font color='gray'>Predictions 🔮
 </font></h2>

In [12]:
result = model.predict(test_x)
result = np.argmax(result,axis = 1)
result = pd.Series(result,name='Label')

875/875 [==============================] - 10s 11ms/step


<h2 align="left"><font color='gray'>Submission 🟰
 </font></h2>

In [13]:
submission = pd.concat([pd.Series(range(1,28001),name = 'ImageId'),result],axis = 1)
submission.to_csv("submission.csv",index = False)